In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jul 17 00:00:09 2022

@author: whh

vx: 15967946943

GUI：进出口贸易额预测model的GUI
"""


from pandas import read_csv, to_datetime, date_range
from numpy import log2, exp2
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import warnings
warnings.filterwarnings('ignore')


import tkinter as tk
from tkinter import *
from tkinter import ttk, IntVar, filedialog
from fbprophet import Prophet


from dateutil.relativedelta import relativedelta
from datetime import datetime
import ctypes



window = tk.Tk()
window.geometry("1000x770")
window.configure(bg='white')
window.resizable(width=False,height=False) 
window.title('贸易额预测')


# 解决屏幕分辨率的问题
ctypes.windll.shcore.SetProcessDpiAwareness(1)
ScaleFactor=ctypes.windll.shcore.GetScaleFactorForDevice(0)
window.tk.call('tk', 'scaling', ScaleFactor/75)



######################################
### 一、创建第一个框架（GUI顶部）
######################################
frame1 = tk.Frame(window, bg='white')  
frame1.place(width=1000, height=70)
tk.Label(frame1, text='宁波口岸进出口贸易额预测', bg='white', font=("SimHei", 15)).pack(pady=0)



######################################
### 二、创建第二个框架（GUI左）
######################################
frame2 = tk.Frame(window)  
frame2.configure(bg='white')
frame2.place(x=0, y=70, width=300,height=600)       


### 选择文件（贸易额.csv）
def UploadAction(event=None):
    filename = filedialog.askopenfilename()
    df_trade = read_csv('%s'%filename, encoding='gbk')  
    df_trade.loc[:,'时间'] = df_trade.loc[:,'时间'].apply(to_datetime)    
    return df_trade


# 初始化变量
global  start_date,end_date
start_date = 144
end_date = 203
date_list=UploadAction()
dates = date_list['时间'].apply(lambda x:x.strftime('%Y-%m-%d')).tolist()
Alldata = date_list



# 1、Button1：选择文件
bt1 = tk.Button(
    frame2, text='选择文件', font=("SimHei", 10),
    width=10, height=2, bd=4, bg='lightsteelblue', 
    command=lambda: UploadAction()
    )
bt1.pack(side=tk.TOP, pady=25)





######################
# 创建二（1）框架（左GUI左）
######################
frame21 = tk.Frame(frame2)  
frame21.configure(bg='white')
frame21.place(x=0, y=120, width=150,height=350)       


# label1：起始日期
lb1 = tk.Label(
    frame21, text='起始日期', font=("SimHei", 10), bg='white',
    width=10, height=1)
lb1.pack(side=tk.TOP, pady=20)


# label2：结束日期
lb2 = tk.Label(
    frame21, text='结束日期', font=("SimHei", 10), bg='white',
    width=10, height=1)
lb2.pack(side=tk.TOP, pady=20)


# label3：预测月数
lb3 = tk.Label(
    frame21, text='预测月数', font=("SimHei", 10), bg='white',
    width=10, height=1)
lb3.pack(side=tk.TOP, pady=20)


# label5：修正系数
lb5 = tk.Label(
    frame21, text='修正系数', font=("SimHei", 10), bg='white',
    width=10, height=1)
lb5.pack(side=tk.TOP, pady=20)


# label4：修正日期
lb4 = tk.Label(
    frame21, text='修正日期', font=("SimHei", 10), bg='white',
    width=10, height=1)
lb4.pack(side=tk.TOP, pady=20)



######################
# 创建二（2）框架（左GUI右）
######################
frame22 = tk.Frame(frame2)  
frame22.configure(bg='white')
frame22.place(x=150, y=120, width=150,height=440)       



# entry1：起始日期
def Start_date(*args):  
    global  start_date
    start_date=list(dates).index(entry1.get())
entry1 = ttk.Combobox(frame22, width=10, font=("SimHei", 10)) 
entry1["values"] = dates
entry1.current(start_date) 
entry1.bind("<<ComboboxSelected>>", Start_date)  # 绑定Start_date()事件.(下拉列表框被选中时，绑定Start_date()函数)
entry1.pack(anchor=tk.CENTER,side=tk.TOP, pady=20)



# entry2：结束日期
def End_date(*args):  
    global  end_date
    end_date = list(dates).index(entry2.get())
    return end_date
entry2 = ttk.Combobox(frame22, width=10, font=("SimHei", 10)) 
entry2["values"] = dates
entry2.current(end_date) 
entry2.bind("<<ComboboxSelected>>", End_date)  # 绑定End_date()事件.(下拉列表框被选中时，绑定End_date()函数)
entry2.pack(anchor=tk.CENTER, side=tk.TOP, pady=20)



# entry3：预测月数
entry3 = tk.Entry(frame22, width=13, font=("SimHei", 10))
entry3.insert(0, 6)   # 预测月份设置初始值：6
entry3.pack(side=tk.TOP, pady=20)



# entry5：修正系数
num_list=[0.8, 0.9, 1.0, 1.1, 1.2]
def Num(*args):  
    global  num
    num = entry5.get()
entry5 = ttk.Combobox(frame22, width=10, font=("SimHei", 10)) 
entry5["values"] = num_list
entry5.current(2)   #指定默认值为1.0
entry5.bind("<<ComboboxSelected>>", Num)  # 绑定Num()事件.(下拉列表框被选中时，绑定Num()函数)
entry5.pack(side=tk.TOP, pady=20)




# Checkbutton4（多选框）：获取后6个月的修正日期（单选/多选/全选 ）
def Get_modify_buttom():
    global  modify_date, month_list
    modify_date = entry2.get()
    
    end_dates = datetime.strptime(modify_date, '%Y-%m-%d')
    end_datess = end_dates - relativedelta(months=-1)
    month_list = date_range(start=end_datess, periods=6, freq='MS')  #pd.date_range(start=end_datess, periods=6, freq='MS')
    
    y_m_d = ['0']
    for i in month_list:
        m1 = i.strftime("%Y{y}%m{m}%d").format(y='-', m='-')
        y_m_d.append(m1)
    dic = dict(zip(months, y_m_d))
    
    # 复选框选择
    def selected():
        m=[]
        for mon in months:            
            if all_vars[mon].get()==1:   m.append(dic.get(mon))
        return m
    
    # 设置全选/单选
    if selected().count('0'):   mm = y_m_d[1:]
    else:    mm = selected()
    return mm



all_vars = {}
months = ["All", '第1个月', '第2个月', '第3个月', '第4个月', '第5个月', '第6个月']
for mon in months:
    var = IntVar()
    entry4 = Checkbutton(
        frame22, text=mon, variable=var, 
        width=8, height=0, bg='white', bd=0, 
        font=("SimHei", 8),justify='left'
        )
    entry4.pack(side=tk.TOP, pady= 2, padx=0)
    entry4.bind("<<ComboboxSelected>>", Get_modify_buttom) # 绑定事件Get_modify_buttom，实时获取修正月份
    all_vars[mon] = var

    

######################################
### 五、创建第五个框架（GUI底层）
######################################
frame5 = tk.Frame(window)  
frame5.configure(bg='white')
frame5.place(width=1000,height=50, x=0, y=690)       #放置框架
tk.Label(frame5,text='浙大宁波理工学院制作', bg='white', font=("SimHei", 8)).pack(anchor=tk.CENTER, pady=10)




def predict_trade(Alldata):
    start_time = datetime.now()        #记录模型开始预测的时间
    
    global x1, x2, x3, x4, x5
    x1 = entry1.get()   #获取开始日期
    x2 = entry2.get()   #获取结束日期
    x3 = entry3.get()   #获取预测月数
    x4 = Get_modify_buttom()   #获取修正日期
    x5 = entry5.get()     #获取修正系数

    
    Alldata.columns = ['ds', 'y']
    start_index = Alldata[Alldata.ds==x1].index.tolist()[0]
    end_index = Alldata[Alldata.ds==x2].index.tolist()[0]
    
    
    Alldata = Alldata.loc[start_index:end_index, :]
    Alldata['y'] = Alldata['y'].apply(log2)                        
    
    
    # 时间格式转换
    Alldata['ds'] = Alldata['ds'].apply(to_datetime)
    train =  Alldata
    prophet = Prophet(seasonality_mode='multiplicative').fit(train)
    
    
    future = prophet.make_future_dataframe(periods=(int(x3)), freq='MS') 
    forecast = prophet.predict(future)
    
    
    def comparison_dataframe(historical, forecast):
        return forecast.set_index('ds')[['yhat']].join(historical.set_index('ds'))
    cmp_df = comparison_dataframe(Alldata, forecast)
    cmp_df = cmp_df.apply(exp2)
    
    
    ###########################
    #添加修正日期对应值 * 修正系数
    cmp_df.loc[cmp_df.index.isin(x4)==True, 'yhat'] = cmp_df[cmp_df.index.isin(x4)==True]['yhat']* float(x5)
    ###########################
    
    
    ######################################
    ### 三、创建第三个框架（GUI右上）
    ######################################
    frame3 = tk.Frame(window)  
    frame3.configure(bg='white')
    frame3.place(x=310, width=690, y=70, height=400)      
 

    global f, a, vl
    f = plt.Figure(figsize=(6, 4), dpi=100)
    vl = FigureCanvasTkAgg(f, frame3)


    a=f.add_subplot(111)
    a.plot(cmp_df[['y']], color = 'b', label=u'真实值')
    a.plot(cmp_df[['yhat']][:-int(x3)], linestyle='--', color = 'b', label=u'训练拟合值')
    a.plot(cmp_df[['yhat']][-int(x3):], linestyle='--', color = 'g', label=u'预测值')


    a.set_title('整体趋势图', fontsize=10, fontproperties="SimHei")
    a.set_ylabel('贸易额（亿美元）', fontsize=10, fontproperties="SimHei")

    
    font1 = {'family': "SimHei",  'size': 8}
    a.legend(prop=font1)
    vl.draw()
    vl.get_tk_widget().pack(anchor=tk.CENTER)


    ######################################
    ### 四、创建第四个框架（GUI右下）
    ######################################
    frame4 = tk.Frame(window)  
    frame4.configure(bg='white')
    frame4.place(x=310, width=690, y=470, height=200)     
    tk.Label(frame4,text='预测结果', font=("SimHei", 9), bg='white').pack(side=tk.TOP, pady=10)
    
    
    columns = ("日期", "贸易额（亿美元）")
    treeview = ttk.Treeview(frame4, height=20, show="headings", columns=columns)  # 表格
    
    
    treeview.heading("日期", text="日期") 
    treeview.heading("贸易额（亿美元）", text='贸易额（亿美元）')
    treeview.column("日期", width=250, anchor='center') 
    treeview.column("贸易额（亿美元）", width=250, anchor='center')
    
    
    d_date = cmp_df[-int(x3):].index.strftime('%Y-%m').tolist()
    v_value = cmp_df[-int(x3):].yhat.tolist()
    
    
    for i in range(min(len(d_date),len(v_value))): # 写入数据
        treeview.insert('', i, values=(d_date[i], round(v_value[i], 2)))
    treeview.pack(anchor=tk.CENTER, padx=40)
    
    
    # 3.8版有bug，要多加的代码！！！------------------------------------------
    #（为实现颜色区分这个需求都快哭死了，再提需求马上就到撒哈拉沙漠找片海溺死去！！！）   
    def tree_color(): # 表格栏隔行显示不同颜色函数
        i=0
        items=treeview.get_children() # 得到根目录所有行的iid
        for hiid in items:
            if i<3: 
                tag1='1-3月' # 前三月
            else:
                tag1='4-6月' # 后三月
            treeview.item(hiid,tag=tag1) 
            i+=1 
        
        
    def fixed_map(option):
        return [elm for elm in style.map("Treeview", query_opt=option)
    if elm[:2] != ("!disabled", "!selected")]
 
    style = ttk.Style()
    style.map("Treeview", foreground=fixed_map("foreground"), background=fixed_map("background"))
    
    #定义背景色风格
    treeview.tag_configure('1-3月', background='lightsteelblue')  #1-3月为浅蓝色背景颜色
    tree_color() 
 

    #预测过程总时间
    time_diff = (datetime.now()-start_time).seconds     
    time = tk.StringVar()
    time.set('预测时间：'+str(time_diff)+'s')
    label = tk.Label(frame5, textvariable=time, bg='white', font=("SimHei", 10))
    label.place(width=200,height=20, x=35, y=0)
    

    


    
######################
# 创建二（2）框架（左GUI底层）
######################
frame23 = tk.Frame(frame2)  
frame23.configure(bg='white')
frame23.place(x=0, y=560, width=310,height=40) 



# Button2：预测
bt2 = tk.Button(
    frame23, text='预测', font=("SimHei", 10), 
    width=7, height=1, bd=2, bg='lightsteelblue', command = lambda: predict_trade(Alldata))
bt2.pack(side=tk.LEFT, padx=30)


# Button3：退出      
bt3 = tk.Button(
    frame23, text='退出', font=("SimHei", 10),
    width=7, height=1, bd=2, bg='lightsteelblue', command = window.destroy)
bt3.pack(side=tk.LEFT, padx=30)



window.mainloop()

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
